# Spread Analysis - C#

## I - Presentation
### I.1 - Overview
As the smallest trade unit in an exchange, understanding the tick size and its effects is important to building efficient and coherent trading strategies.

Through this notebook, we will study the spread at the top of the book on the US market, Ask-Bid.
The main objective is to visualize the bid-ask spread value and percentage based on the vwap price for every ticker studied.
Our stock data will be restrained to the primary market XN and their equivalent in the secondary market BATS.

### I.2 - Services used
This sample uses gRPC requests in order to retrieve Spread data from the hosted service. The queried endpoint in this script are:
* *TickSpreadService*: to directly retrieve spread data reponse from the server

### I.3 - Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
    * *Systemathics.Apis.Services.Indices.V1*
    * *Systemathics.Apis.Services.StaticData.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

### I.4 - Notebook Preparations
#### Packages and Import

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: CsvHelper"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using Systemathics.Apis.Services.Indices.V1;
using Systemathics.Apis.Services.StaticData.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Core;
using Grpc.Net.Client;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;
// Import/export csv files
using CsvHelper;
using System.IO;
using System.Globalization;

#### Retreive Authentification Token
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

## II - Exchange Components

### II.1 - Parameters

In [ ]:
// Set exchange
var exchanges = new List<string>(){"XNGS","XNYS","XNCM","XNMS"};
var bats = false;

var name = "XN";
if (bats){name = "BATS";}

In [ ]:
public class Instrument
{
    public string Ticker{get; set;}
    public string Exchange{get; set;}
    public string Operating{get; set;}
    public string Primary{get; set;}
    public string Name{get; set;}
    public string Isin{get; set;}
}

### II.2 - Request

In [ ]:
// Create a class to handle the instrument's mapping codes
var instruments = new List<Instrument>();
var service = new StaticDataService.StaticDataServiceClient(channel);
for (var i = 0; i < exchanges.Count ;i++) 
    {
    // Generate request
    var request = new StaticDataRequest { AssetType = AssetType.Equity, Exchange = exchanges[i], Count = 3000};

    // Call the service
    var reply = service.StaticData(request, headers);
    // display(reply.Equities.Count);
    // Get mapping codes for the requested instruments
    foreach (var rep in reply.Equities) 
    {
        // Get mapping codes
        var ticker = rep.Identifier.Ticker;
        rep.Mapping.TryGetValue("Bloomberg", out var bloomberg);
        
        if (rep.Identifier.Exchange == "BATS")
        {  
            //instruments.Add(new Instrument(){Ticker = ticker,Exchange = rep.Identifier.Exchange,Operating = rep.Operating, Primary = rep.Primary, Name = rep.Name, Isin = rep.Isin});
        }
        else
        {
            instruments.Add(new Instrument(){Ticker = ticker,Exchange = rep.Identifier.Exchange,Operating = rep.Operating, Primary = rep.Primary, Name = rep.Name, Isin = rep.Isin});
        }
    };
};

### II.3 - Import / Export

#### Export

In [ ]:
using (var writer = new StreamWriter($"US_{name}.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture))
{
    csv.WriteRecords(instruments);
}

#### Import

## III - Spread Study

### III.1 - Parameters

#### Storage

In [ ]:
// Input trade to load from the csv file
public class TickerData
{
    public string Ticker { get; set; }
    public string Exchange { get; set; }
    public string Operating { get; set; }
    public string Primary { get; set; }
    
    public double BidLast { get; set; }
    public double BidAverage { get; set; }
    public double BidVwap { get; set; }
    public double BidMin { get; set; }
    public double BidMax { get; set; }
    public long BidVolume { get; set; }
    
    public double AskLast { get; set; }
    public double AskAverage { get; set; }
    public double AskVwap { get; set; }
    public double AskMin { get; set; }
    public double AskMax { get; set; }
    public long AskVolume { get; set; }
    
    public double Last { get; set; }
    public double Average { get; set; }
    public double Min { get; set; }
    public double Max { get; set; }
    public long Count { get; set; }
    public long Skipped { get; set; }
}

#### Constraints

In [ ]:
var today = DateTime.Today;
var dateIntervals = new DateInterval
{
    StartDate = new Date { Year = 2022, Month = 02, Day = 18 },
    EndDate = new Date { Year = 2022, Month = 02, Day = 18 }
};

var timeInterval = new TimeInterval
{
    StartTime = new TimeOfDay { Hours = 19, Minutes = 00, Seconds = 00 },
    EndTime = new TimeOfDay { Hours = 20, Minutes = 00, Seconds = 00 }
};

In [ ]:
// Generate constraints based on the previous time selection:
var my_constraints = new Constraints();
my_constraints.DateIntervals.Add(dateIntervals);
my_constraints.TimeIntervals.Add(timeInterval); 

### III.2 - Request

In [ ]:
// Instantiate tick VWAP service
var service = new TickSpreadService.TickSpreadServiceClient(channel);

In [ ]:
#!time
using (var writer = new StreamWriter($"spread_US_{name}_19002000.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture))
{
    foreach (var t in instruments)
    {
        var excluded = new List<string>(){"FUNC","TWOA","LHC", "SPGS", "CCV","HERA","NYXH"};
        if (excluded.Contains(t.Ticker) && bats) {continue;}
        
        var request = new TickSpreadRequest() 
        { 
            Identifier = new Identifier { Exchange = t.Exchange, Ticker = t.Ticker}, 
            Constraints = my_constraints,
            MinAccepted = 0,
            MaxAccepted = 5
        };

        var call = service.TickSpread(request, headers);
        var spreadData = new List<TickerData>{};
        
        await foreach (var data in call.ResponseStream.ReadAllAsync())
        {
            spreadData.Add(new TickerData{
                Ticker = t.Ticker,
                Exchange = t.Exchange,
                Operating = t.Operating,
                Primary = t.Primary,

                BidLast = data.Bid.Last,
                BidAverage = data.Bid.Average,
                BidVwap = data.Bid.Vwap,
                BidMin = data.Bid.Min,
                BidMax = data.Bid.Max,
                BidVolume = data.Bid.Volume,

                AskLast = data.Ask.Last,
                AskAverage = data.Ask.Average,
                AskVwap = data.Ask.Vwap,
                AskMin = data.Ask.Min,
                AskMax = data.Ask.Max,
                AskVolume = data.Ask.Volume,

                Last = data.Last,
                Average = data.Average,
                Min = data.Min,
                Max = data.Max,
                Count = data.Count,
                Skipped = data.Skipped
                });
        };
        csv.WriteRecords(spreadData);
    };
};


### III.3 - Import